**Problem statement:** 

Continuous bag of words (cbow) word2vec word embedding work is that it tends to predict the probability of a word given a context. A context may be a single word or a group of words. But for simplicity, I will take a single context word and try to predict a single target word. 

The purpose of this assignment is to be able to create a word embedding for the  given data set.  

**Data set :** w2v.txt 

### Importing libraries

In [1]:
import itertools
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

In [2]:
# Drawing the embeddings
import matplotlib.pyplot as plt

# Deep learning: 
from keras.models import Input, Model
from keras.layers import Dense

from scipy import sparse

In [3]:
# Custom functions
from utility import text_preprocessing, create_unique_word_dict

In [ ]:
# Reading the text from the input folder
texts = pd.read_csv('input/sample.csv')
texts = [x for x in texts['text']]

### Function Definations

In [2]:
# Finding the maximum length of an array required to include all sentences. Vocabulary size, in order to create the 
# same size inputs by padding with zeros

def get_max_vocabulary_size(encoded_line_words):
    max_voc_size = 0
    max_integer_index = 0
    for l in encoded_line_words:
        if (len(l) > 0) and (np.argmax(l) > max_integer_index):
            max_integer_index = l[np.argmax(l)]
        if max_voc_size < len(l):
            max_voc_size = len(l)
    return max_voc_size

In [3]:
# Creating a new list with the targets for the words with window size as window_size. 
# Only positive window size is taken into consideration

def map_target(context, window_size):
    if window_size < 0:
        print("Only Positive window sie is expected.")
        return;
    target = context[window_size:]
    m = 10
    i = 0
    while i < window_size :
        target = np.append(target, [0])
        i = i + 1
    return np.array(target)

In [4]:
# Creating array of targets using the map_target function

def map_all_targets(context_list, window_size):
    targets = []
    for context in context_list:
        target_row = map_target(context, window_size)
        targets.append(np.array(target_row))
    return targets

In [5]:
def pad_sequences_words(file_lines_words, maxlen):
    padded_word = []
    for line_word in file_lines_words:
        blank_word_index = len(line_word)
        while blank_word_index < maxlen:
            line_word.append("")
            padded_word.append(line_word)
            blank_word_index = blank_word_index + 1

In [6]:
# Read the text file and return a dict with lines of the file, words in line, 
# encoded words in line, padded context words and targets

def read_txt(txt_file, window_size):
    file_lines = []
    file_lines_words = []
    encoded_file_lines_words = []
    target_file_lines_words = []
    with open(txt_file, 'r', encoding="utf-8") as file:
        try:
            for line in file:
                try:
                    line = line.strip()
                    if line!="":
                        file_lines.append(line)
                        allWords = line.split(" ")
                        file_words = []
                        encoded_file_words = []
                        try:
                            for word in allWords:
                                if word.strip()!="" and len(allWords)>1:
                                    file_words.append(word.strip())  
                                    encoded_file_words.append(one_hot(word.strip(),len(allWords))[0])
                        except Exception as E:
                            print ("got An exception 1: ", E)
                            pass  
                        file_lines_words.append(file_words)
                        encoded_file_lines_words.append(encoded_file_words)
                except Exception as E:
                    print ("got An exception 2: ", E)
                    pass         
        except Exception as E:
            print ("got An exception 3: ", E)
            pass  
        
    max_integer_index = get_max_vocabulary_size(encoded_file_lines_words)
    padded_context = pad_sequences(encoded_file_lines_words, maxlen = max_integer_index, padding='post')
    padded_context_words = pad_sequences_words(file_lines_words, maxlen = max_integer_index)
    targets = np.array(map_all_targets(context_list=padded_context, window_size=window_size))
    return {'lines':file_lines, 'line_words':file_lines_words, 'encoded_line_words':encoded_file_lines_words, 
            'padded_context':padded_context, 'targets':targets, 'vocab_size': max_integer_index}

In [7]:
# All the parameters calculation and constants are being set using this function. 
# Any changes to the parameters should be mafe here to avoid execution errors.

def parameters(context):
    INPUT_DIM = max([max(ctx) for ctx in context])+1
    OUTPUT_DIM = 8
    INPUT_LENGTH = len(context[0])
    EPOCHS = 1000
    VERBOSE = 1
    LOSS = 'categorical_crossentropy'
    ACTIVATION = 'softmax'
    OPTIMIZER = 'adam'
    MATRIX = ['accuracy']
    BESTMODEL = 'embeddings.h5'
    return (INPUT_DIM, OUTPUT_DIM, INPUT_LENGTH, EPOCHS, VERBOSE, LOSS, ACTIVATION, OPTIMIZER, MATRIX, BESTMODEL)

In [8]:
file_details = read_txt("w2v.txt", window_size=1)

In [9]:
context = file_details['padded_context']
targets = file_details['targets']

In [10]:
# Confirming that the shapes of context and targets are same

context.shape, targets.shape

((13, 178), (13, 178))

In [11]:
# Unit testing the context and it's relavent target. Change the index between 0 to 12 to check the pair

index = 1
context[index], targets[index]

(array([173, 131,  72, 118,  11, 141, 172,  84, 151,   3, 173, 139, 115,
        173,  71,  72, 173,  99,  28,  79, 132, 172, 163,   8, 173,  17,
         72,  58, 140, 161, 150,  74,  19,  77,  44, 173, 133,  72,  51,
        111, 163,   3, 173,  45,   7,  42, 104,  38,  74, 175,   3,  34,
         69,  72,  47, 173,  24,  41,  25,  52,  44, 100, 173, 163, 126,
         72,  11,  46,   3,   5, 159,  13,  34, 141,  23, 132,  11, 104,
         38,  74, 171, 115,  14, 165,   3, 170, 140, 166,  84,   3,  32,
         11, 144,  55, 141, 172,  72, 112,  11, 132,  16,   3,  32,  11,
        144,  55, 141, 172,  72,  96,   5,   3,  32, 173,  11,  38,  26,
        140, 166, 144,  92,   3,  72,  38,  74, 163, 108, 132,  22,  20,
          3,  77,  52,  44,  97, 103, 124, 173,  12,  44, 145,  46,  72,
        163, 170, 162, 147, 173,  13,  68, 144, 132,  56,   8, 136, 151,
         32, 163, 151, 162, 173, 111,  14, 146, 152,  19, 137, 173, 163,
         72, 172, 132, 173,  84, 114,  72, 173,  79

In [12]:
INPUT_DIM, OUTPUT_DIM, INPUT_LENGTH, EPOCHS, VERBOSE, LOSS, ACTIVATION, OPTIMIZER, MATRIX, BESTMODEL = \
parameters(context)

In [13]:
# Calback array to save the best model. Best model is decided on the basis of accuracy.

callback_list = [tf.keras.callbacks.ModelCheckpoint(filepath=BESTMODEL, monitor = MATRIX[0], 
                                                            save_best_only=True)]

<IPython.core.display.Javascript object>

In [14]:
# Creating the sequential model with Embedding layer with output dimention of 8
# The weights of the first layers will be used as the word embedding
# these weights will later be used to calculate the distance between the words.
# least distant words are most related and vice versa

model = Sequential()
embedding_layer = Embedding(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM, input_length=INPUT_LENGTH)
model.add(embedding_layer)
model.add(Flatten())

# A softmax activation is used.

model.add(Dense(INPUT_LENGTH, activation = ACTIVATION))

model.compile(optimizer = OPTIMIZER, loss=LOSS, metrics = MATRIX)

2022-04-24 21:36:11.479587: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
# Summary of the model 

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 178, 8)            1408      
_________________________________________________________________
flatten (Flatten)            (None, 1424)              0         
_________________________________________________________________
dense (Dense)                (None, 178)               253650    
Total params: 255,058
Trainable params: 255,058
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
# Training cell. History variable contains all the loss and accuracy at each step.

history = model.fit(context, targets, epochs = EPOCHS, verbose=VERBOSE, callbacks=callback_list)

Epoch 1/1000


2022-04-24 21:36:11.771477: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 [==============================] - 1s 683ms/step - loss: 10237.0361 - accuracy: 0.0000e+00
Epoch 2/1000
1/1 [==============================] - 0s 7ms/step - loss: 10221.8135 - accuracy: 0.0000e+00
Epoch 3/1000
1/1 [==============================] - 0s 10ms/step - loss: 10206.7539 - accuracy: 0.0000e+00
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 10191.6348 - accuracy: 0.0000e+00
Epoch 5/1000
1/1 [==============================] - 0s 18ms/step - loss: 10176.3369 - accuracy: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 16ms/step - loss: 10160.7979 - accuracy: 0.0000e+00
Epoch 7/1000
1/1 [==============================] - 0s 23ms/step - loss: 10144.9902 - accuracy: 0.0000e+00
Epoch 8/1000
1/1 [==============================] - 0s 26ms/step - loss: 10128.8926 - accuracy: 0.0000e+00
Epoch 9/1000
1/1 [==============================] - 0s 44ms/step - loss: 10112.5068 - accuracy: 0.0000e+00
Epoch 10/1000
1/1 [==============================]

1/1 [==============================] - 0s 5ms/step - loss: 33965.8906 - accuracy: 0.0769
Epoch 157/1000
1/1 [==============================] - 0s 8ms/step - loss: 34348.8906 - accuracy: 0.0769
Epoch 158/1000
1/1 [==============================] - 0s 8ms/step - loss: 34734.7109 - accuracy: 0.0769
Epoch 159/1000
1/1 [==============================] - 0s 8ms/step - loss: 35123.3555 - accuracy: 0.0769
Epoch 160/1000
1/1 [==============================] - 0s 6ms/step - loss: 35514.8203 - accuracy: 0.0769
Epoch 161/1000
1/1 [==============================] - 0s 6ms/step - loss: 35909.1055 - accuracy: 0.0769
Epoch 162/1000
1/1 [==============================] - 0s 9ms/step - loss: 36306.2109 - accuracy: 0.0769
Epoch 163/1000
1/1 [==============================] - 0s 6ms/step - loss: 36706.1289 - accuracy: 0.0769
Epoch 164/1000
1/1 [==============================] - 0s 6ms/step - loss: 37108.8672 - accuracy: 0.0769
Epoch 165/1000
1/1 [==============================] - 0s 9ms/step - loss: 37514

1/1 [==============================] - 0s 7ms/step - loss: 126586.6719 - accuracy: 0.0769
Epoch 312/1000
1/1 [==============================] - 0s 6ms/step - loss: 127399.6797 - accuracy: 0.0769
Epoch 313/1000
1/1 [==============================] - 0s 7ms/step - loss: 128215.4609 - accuracy: 0.0769
Epoch 314/1000
1/1 [==============================] - 0s 6ms/step - loss: 129033.9453 - accuracy: 0.0769
Epoch 315/1000
1/1 [==============================] - 0s 6ms/step - loss: 129855.1953 - accuracy: 0.0769
Epoch 316/1000
1/1 [==============================] - 0s 7ms/step - loss: 130679.1719 - accuracy: 0.0769
Epoch 317/1000
1/1 [==============================] - 0s 7ms/step - loss: 131505.9375 - accuracy: 0.0769
Epoch 318/1000
1/1 [==============================] - 0s 7ms/step - loss: 132335.4219 - accuracy: 0.0769
Epoch 319/1000
1/1 [==============================] - 0s 6ms/step - loss: 133167.6719 - accuracy: 0.0769
Epoch 320/1000
1/1 [==============================] - 0s 8ms/step - lo

1/1 [==============================] - 0s 6ms/step - loss: 284988.5625 - accuracy: 0.0769
Epoch 467/1000
1/1 [==============================] - 0s 7ms/step - loss: 286218.3438 - accuracy: 0.0769
Epoch 468/1000
1/1 [==============================] - 0s 6ms/step - loss: 287450.7188 - accuracy: 0.0769
Epoch 469/1000
1/1 [==============================] - 0s 6ms/step - loss: 288685.6250 - accuracy: 0.0769
Epoch 470/1000
1/1 [==============================] - 0s 6ms/step - loss: 289923.2188 - accuracy: 0.0769
Epoch 471/1000
1/1 [==============================] - 0s 7ms/step - loss: 291163.4375 - accuracy: 0.0769
Epoch 472/1000
1/1 [==============================] - 0s 6ms/step - loss: 292406.1562 - accuracy: 0.0769
Epoch 473/1000
1/1 [==============================] - 0s 8ms/step - loss: 293651.5000 - accuracy: 0.0769
Epoch 474/1000
1/1 [==============================] - 0s 7ms/step - loss: 294899.5000 - accuracy: 0.0769
Epoch 475/1000
1/1 [==============================] - 0s 6ms/step - lo

1/1 [==============================] - 0s 7ms/step - loss: 505992.5000 - accuracy: 0.0769
Epoch 622/1000
1/1 [==============================] - 0s 6ms/step - loss: 507613.1250 - accuracy: 0.0769
Epoch 623/1000
1/1 [==============================] - 0s 7ms/step - loss: 509236.1562 - accuracy: 0.0769
Epoch 624/1000
1/1 [==============================] - 0s 8ms/step - loss: 510861.6875 - accuracy: 0.0769
Epoch 625/1000
1/1 [==============================] - 0s 5ms/step - loss: 512489.7812 - accuracy: 0.0769
Epoch 626/1000
1/1 [==============================] - 0s 4ms/step - loss: 514120.1875 - accuracy: 0.0769
Epoch 627/1000
1/1 [==============================] - 0s 5ms/step - loss: 515753.0625 - accuracy: 0.0769
Epoch 628/1000
1/1 [==============================] - 0s 7ms/step - loss: 517388.4688 - accuracy: 0.0769
Epoch 629/1000
1/1 [==============================] - 0s 6ms/step - loss: 519026.2812 - accuracy: 0.0769
Epoch 630/1000
1/1 [==============================] - 0s 6ms/step - lo

1/1 [==============================] - 0s 5ms/step - loss: 639696.2500 - accuracy: 0.0769
Epoch 700/1000
1/1 [==============================] - 0s 7ms/step - loss: 641505.3125 - accuracy: 0.0769
Epoch 701/1000
1/1 [==============================] - 0s 5ms/step - loss: 643316.9375 - accuracy: 0.0769
Epoch 702/1000
1/1 [==============================] - 0s 5ms/step - loss: 645130.8125 - accuracy: 0.0769
Epoch 703/1000
1/1 [==============================] - 0s 5ms/step - loss: 646947.1250 - accuracy: 0.0769
Epoch 704/1000
1/1 [==============================] - 0s 5ms/step - loss: 648765.6875 - accuracy: 0.0769
Epoch 705/1000
1/1 [==============================] - 0s 5ms/step - loss: 650586.7500 - accuracy: 0.0769
Epoch 706/1000
1/1 [==============================] - 0s 6ms/step - loss: 652410.2500 - accuracy: 0.0769
Epoch 707/1000
1/1 [==============================] - 0s 5ms/step - loss: 654236.0625 - accuracy: 0.0769
Epoch 708/1000
1/1 [==============================] - 0s 7ms/step - lo

1/1 [==============================] - 0s 8ms/step - loss: 948066.9375 - accuracy: 0.0769
Epoch 855/1000
1/1 [==============================] - 0s 6ms/step - loss: 950236.0000 - accuracy: 0.0769
Epoch 856/1000
1/1 [==============================] - 0s 9ms/step - loss: 952407.6875 - accuracy: 0.0769
Epoch 857/1000
1/1 [==============================] - 0s 8ms/step - loss: 954581.3750 - accuracy: 0.0769
Epoch 858/1000
1/1 [==============================] - 0s 6ms/step - loss: 956757.3750 - accuracy: 0.0769
Epoch 859/1000
1/1 [==============================] - 0s 8ms/step - loss: 958935.6875 - accuracy: 0.0769
Epoch 860/1000
1/1 [==============================] - 0s 11ms/step - loss: 961116.1250 - accuracy: 0.0769
Epoch 861/1000
1/1 [==============================] - 0s 7ms/step - loss: 963299.0625 - accuracy: 0.0769
Epoch 862/1000
1/1 [==============================] - ETA: 0s - loss: 965484.1250 - accuracy: 0.076 - 0s 10ms/step - loss: 965484.1250 - accuracy: 0.0769
Epoch 863/1000
1/1 [

1/1 [==============================] - 0s 5ms/step - loss: 1121682.1250 - accuracy: 0.0769
Epoch 932/1000
1/1 [==============================] - 0s 5ms/step - loss: 1124023.5000 - accuracy: 0.0769
Epoch 933/1000
1/1 [==============================] - 0s 6ms/step - loss: 1126367.7500 - accuracy: 0.0769
Epoch 934/1000
1/1 [==============================] - 0s 5ms/step - loss: 1128714.1250 - accuracy: 0.0769
Epoch 935/1000
1/1 [==============================] - 0s 7ms/step - loss: 1131062.6250 - accuracy: 0.0769
Epoch 936/1000
1/1 [==============================] - 0s 6ms/step - loss: 1133413.2500 - accuracy: 0.0769
Epoch 937/1000
1/1 [==============================] - 0s 6ms/step - loss: 1135765.8750 - accuracy: 0.0769
Epoch 938/1000
1/1 [==============================] - 0s 7ms/step - loss: 1138121.1250 - accuracy: 0.0769
Epoch 939/1000
1/1 [==============================] - 0s 7ms/step - loss: 1140478.2500 - accuracy: 0.0769
Epoch 940/1000
1/1 [==============================] - 0s 6ms/

In [17]:
# The variable word_embeddings contains all the embeddings

word_embeddings = embedding_layer.get_weights()[0]

In [18]:
# Get the searched line and word indeces

def get_indices(word):
    line_index = 0
    word_index = 0
    for line in file_details['line_words']:
        for wrd in line:
            if wrd==word:
                return (line_index, word_index)
            word_index = word_index + 1
        line_index = line_index + 1
    return (-1,-1)

def eq(embd1, embd2):
    isEq = True
    index = 0
    for em in embd1:
        if str(em)!=str(embd2[index]):
            isEq = False
            break
        index = index + 1
    return isEq
        
def get_ohe_indices(embedding):
    ONE = -1
    ind = 0
    for embd in word_embeddings:
        if eq(list(embedding), list(embd)):
            ONE = ind
            break
        ind = ind + 1
    
    line_index = 0
    word_index = 0
    for line in file_details['encoded_line_words']:
        for wrd in line:
            if wrd==ONE:
                return (line_index, word_index)
            word_index = word_index + 1
        line_index = line_index + 1
    return (-1,-1)

def get_word(indices):
    word = '__NoWordFound__'
    if indices!=(-1,-1):
        word = file_details['line_words'][indices[0]][indices[1]]
    return word

In [19]:
get_ohe_indices([ 1.6754463, -1.5663351,  1.6041586,  1.5992835, -1.6179308,
       -1.5989335,  1.6009173, -1.6389836])

(-1, -1)

In [20]:
get_word(get_ohe_indices([ 1.6754463, -1.5663351,  1.6041586,  1.5992835, -1.6179308,
       -1.5989335,  1.6009173, -1.6389836]))

'__NoWordFound__'

In [21]:
# Fetch the word embeddings

def fetch_embedding(indices, word_embeddings):
    # Get the One Hote Encoding of the word using the indices 
    # and passing it to the word embeddings to find the embedding of the word
    print(indices)
    OHE = file_details['encoded_line_words'][indices[0]][indices[1]]
    return word_embeddings[OHE]

In [22]:
len(file_details['encoded_line_words'][1])

178

#### Now the embedding of a word can be fetched in the following way

In [23]:
word = 'In'
indices = get_indices(word=word)
fetch_embedding(indices=indices, word_embeddings=word_embeddings)

(0, 0)


array([-1.602085 , -1.5809002,  1.5815208,  1.6116804,  1.6400144,
       -1.5962365, -1.6054143,  1.5970465], dtype=float32)

In [24]:
# Function to find the least distant word with the help of word_embeddings. 
# Here the above sample of code will be leveraged to find the embedding of 
# the two words and calculate the cosine theta distance. Let us make all possible pairs and store the distances.
# Later the least distance index can be found to map with the closest pair.
# given a word and the learned word_embeddings, this function will return the next (closest) word.
# evaluation_method is cosine for the sake of simplicity

def closest_word(word, word_embeddings):
    word_pairs = []
    word_pairs_distances = []
    line_index, word_index = get_indices(word)
    print(word_embeddings.shape)
    if (line_index == -1) or (word_index == -1):
        print("searched word not found in the given context!")
    else:
        indices = get_indices(word=word)
        embedding = fetch_embedding(indices=indices, word_embeddings=word_embeddings)
        try:
            for embd in word_embeddings:
                result = 1 - spatial.distance.cosine(embedding, embd)
                embd_wd = get_word(get_ohe_indices(embd))
                word_pairs.append(word+':'+embd_wd)
                word_pairs_distances.append(result)
                print(result, line_index, word_index)
        except Exception as E:
            print ("got An exception 1: ", E)
            pass 
    return {'pairs':word_pairs, 'distance':word_pairs_distances}

In [25]:
closest_word("In", word_embeddings)

(176, 8)
(0, 0)
-0.24692845344543457 0 0
0.001006013248115778 0 0
got An exception 1:  list index out of range


{'pairs': ['In:__NoWordFound__', 'In:samples'],
 'distance': [-0.24692845344543457, 0.001006013248115778]}